In [ ]:
import requests
import pandas as pd

req = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?limit=20")
wb = req.json()

url_belibstat = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-donnees-statiques/records?limit=20"

url_count = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records?limit=20"

url_belibdyn = "https://parisdata.opendatasoft.com/api/explore/v2.1/catalog/datasets/belib-points-de-recharge-pour-vehicules-electriques-disponibilite-temps-reel/records?limit=20"

In [10]:
print(req.content[:1000])

b'{"total_count": 1947, "results": [{"id_pdc_local": "FR*V75*E9015*06*2", "statut_pdc": "En service", "arrondissement": "15e Arrondissement", "id_station_local": "FR*V75*P9015*06", "id_station_itinerance": "FRV75P901506", "nom_station": "Paris | Rue Desnouettes 55", "code_insee_commune": "75115", "implantation_station": "Voirie", "nbre_pdc": "3", "date_maj": "2024-07-01", "condition_acces": "Acc\\u00e8s libre", "adresse_station": "55 Rue Desnouettes 75015 Paris", "coordonneesxy": {"lon": 2.286912, "lat": 48.83571}, "gratuit": "false", "paiement_acte": "true", "paiement_cb": "false", "paiement_autre": "true", "reservation": "false", "observations": ["Paiement autre - badge RFID Belib ou autre, application Belib, QRCode sur chaque PDC, site internet Belib", "Plus d\'information - https://belib.paris.fr"], "num_pdl": "07436179348549", "siren_amenageur": "531680445", "contact_amenageur": "assistance.technique@tevgo.fr", "nom_amenageur": "TOTALENERGIES", "nom_operateur": "TOTALENERGIES", "c

In [12]:
import pandas as pd
import geopandas as gpd


def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    df = pd.json_normalize(wb["results"])

    

    return df


In [18]:
dpe_belibstat = get_dpe_from_url(url_belibstat)
dpe_count = get_dpe_from_url(url_count)
dpe_belibdyn = get_dpe_from_url(url_belibdyn)
dpe_belibstat.head(2)



,id_pdc_local,statut_pdc,arrondissement,id_station_local,id_station_itinerance,nom_station,code_insee_commune,implantation_station,nbre_pdc,date_maj,...,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,prise_type_3,horaires,raccordement,coordonneesxy.lon,coordonneesxy.lat
0,FR*V75*E9015*06*2,En service,15e Arrondissement,FR*V75*P9015*06,FRV75P901506,Paris | Rue Desnouettes 55,75115,Voirie,3,2024-07-01,...,True,False,False,False,False,False,24/7,Direct,2.286912,48.835710
1,FR*V75*E9015*07*3,En service,15e Arrondissement,FR*V75*P9015*07,FRV75P901507,Paris | Rue Theophraste Renaudot 21,75115,Voirie,3,2024-07-01,...,True,True,True,True,False,True,24/7,Direct,2.296744,48.841286


In [19]:
dpe_count.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d.lon,geo_point_2d.lat,geo_shape.type,geo_shape.geometry.coordinates,geo_shape.geometry.type
0,6529,Pyrenees,2023-11-06T02:00:00+00:00,49.0,NaN,Inconnu,3384,Pyrenees-La_Mare,3377,Belleville-Pyrenees,Invalide,2003-03-11,2023-01-01,2.386763,48.873903,Feature,"[[2.388005680304629, 48.87384643458903], [2.38...",LineString
1,6529,Pyrenees,2023-11-06T01:00:00+00:00,76.0,NaN,Inconnu,3384,Pyrenees-La_Mare,3377,Belleville-Pyrenees,Invalide,2003-03-11,2023-01-01,2.386763,48.873903,Feature,"[[2.388005680304629, 48.87384643458903], [2.38...",LineString


In [20]:
dpe_belibdyn.head(2)

,id_pdc,statut_pdc,url_description_pdc,last_updated,adresse_station,code_insee_commune,arrondissement,coordonneesxy.lon,coordonneesxy.lat,coordonneesxy
0,FR*V75*EPX11*05*4,Disponible,https://parisdata.opendatasoft.com/explore/dat...,2022-06-13T02:30:01+00:00,8 Boulevard Voltaire 75011 Paris,75111,11e Arrondissement,2.366438,48.865765,NaN
1,FR*V75*EPX13*05*1,Inconnu,https://parisdata.opendatasoft.com/explore/dat...,2023-06-06T02:30:04+00:00,188 Avenue de Choisy 75013 Paris,75113,13e Arrondissement,2.356917,48.830390,NaN
